In [ ]:
import os
from docx import Document
from docx.text.paragraph import Paragraph
from docx.table import _Cell

def replace_text_in_runs(container, old_text, new_text):
    """
    Replaces text within the runs of a container (Paragraph or _Cell), preserving formatting.

    Args:
        container (Paragraph or _Cell): The container object (paragraph or table cell).
        old_text (str): The text to search for.
        new_text (str): The text to replace with.

    Returns:
        int: The number of replacements made within this container.

    Note: This basic version handles replacements *within* single runs.
          Replacing text that *spans* multiple runs is significantly more complex
          and might require deleting/inserting runs carefully.
    """
    count = 0
    if not hasattr(container, 'paragraphs'): # Handle direct paragraphs
         paragraphs = [container] if isinstance(container, Paragraph) else []
    else: # Handle containers like _Cell which have paragraphs
         paragraphs = container.paragraphs

    for paragraph in paragraphs:
        # Workaround for simple replacements within runs:
        # We build the text and check if the old_text is present.
        # A more robust solution would track text across run boundaries.
        inline_runs_text = "".join(r.text for r in paragraph.runs)

        if old_text in inline_runs_text:
            # Simple approach for replacement contained within single runs:
            for run in paragraph.runs:
                if old_text in run.text:
                    # Perform replacement directly in the run
                    run.text = run.text.replace(old_text, new_text)
                    count += 1
                    # Note: If old_text appears multiple times in the run,
                    # this replaces all occurrences within that run.

            # --- More Complex Scenario (Text Spanning Runs) ---
            # Handling text spanning multiple runs requires a more complex logic:
            # 1. Collect text and run indices: [(run_index, run_text), ...]
            # 2. Find start/end run indices covering old_text in the concatenated text.
            # 3. Clear text from runs between start+1 and end-1.
            # 4. Modify text in start and end runs carefully.
            # 5. If new_text has different formatting needs, you might need to insert new runs.
            # This simplified example does *not* implement the complex spanning logic.
            # print(f"Warning: Replacement for '{old_text}' might be complex if it spans multiple formatting runs.")

    return count


def edit_section_in_word(docx_path, section_identifier, old_text, new_text, identifier_type="text"):
    """
    Edits text within a defined section of a Word document, preserving formatting.

    Args:
        docx_path (str): Path to the input/output Word document.
        section_identifier (str): The text, style name, or marker identifying the section.
        old_text (str): The text to replace within the identified section.
        new_text (str): The text to replace with.
        identifier_type (str): How to identify the section:
                               'text' (default): Paragraph contains section_identifier text.
                               'style': Paragraph has style name section_identifier.
                               'table': Process all tables (section_identifier ignored).
                               # Could add 'between_markers' etc.
    """
    if not os.path.exists(docx_path):
        print(f"Error: Document not found at '{docx_path}'")
        return

    document = Document(docx_path)
    total_replacements = 0
    in_target_section = (identifier_type != "style" and identifier_type != "text") # Start true if not matching paragraphs

    print(f"Processing document: {docx_path}")
    print(f"Looking for section identifier '{section_identifier}' (type: {identifier_type})")
    print(f"Replacing '{old_text}' with '{new_text}'")

    # Iterate through main body paragraphs
    for para in document.paragraphs:
        process_paragraph = False
        if identifier_type == "text":
            if section_identifier in para.text:
                # Simple: consider this paragraph as the section (or part of it)
                process_paragraph = True
        elif identifier_type == "style":
            if para.style and para.style.name == section_identifier:
                process_paragraph = True
        else:
             process_paragraph = True # If not text/style based, assume always process (e.g. for tables)


        if process_paragraph:
             total_replacements += replace_text_in_runs(para, old_text, new_text)

    # Iterate through tables
    # if identifier_type == "table": # Or process tables regardless? Adjust logic as needed.
    print("\nProcessing tables...")
    table_replacements = 0
    for table in document.tables:
        for row in table.rows:
            for cell in row.cells:
                # Cells contain paragraphs, process runs within cell paragraphs
                table_replacements += replace_text_in_runs(cell, old_text, new_text) # Pass the cell object
    if table_replacements > 0:
        print(f"Made {table_replacements} replacements in tables.")
        total_replacements += table_replacements


    if total_replacements > 0:
        try:
            document.save(docx_path)
            print(f"\nSuccessfully saved changes to '{docx_path}'. Total replacements: {total_replacements}")
        except Exception as e:
            print(f"\nError saving document: {e}. Make sure the file is not open elsewhere.")
    else:
        print("\nNo instances of the text to replace were found in the specified sections.")


# --- Example Usage ---
doc_file = "template.docx"  # Use the Word doc created earlier or any other .docx
section_id = "RICHARD"       # Example: Target paragraphs containing this word
text_to_find = "RICHARD"
text_to_replace_with = "HI"

# Edit paragraphs containing "Introduction"
edit_section_in_word(doc_file, section_id, text_to_find, text_to_replace_with, identifier_type="text")

# Example: Edit paragraphs with a specific style
# section_id_style = "Heading 2"
# text_to_find_style = "outdated term"
# text_to_replace_style = "current terminology"
# edit_section_in_word(doc_file, section_id_style, text_to_find_style, text_to_replace_style, identifier_type="style")

# Example: Edit text throughout all tables
# text_to_find_table = "Table Data Point X"
# text_to_replace_table = "Table Data Point Y"
# edit_section_in_word(doc_file, "ignored_for_table_mode", text_to_find_table, text_to_replace_table, identifier_type="table") # Identifier ignored here